In [ ]:
# Install spaCy if not already installed
!pip install -U spacy

# Download spaCy English model
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
import re

In [2]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Update the file path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/AAPL Financial Report.xlsx'

In [4]:
# Load the financial dataset from Google Drive
financial_data = pd.read_excel(file_path, index_col=0)
financial_data

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009
FT / Year,,,,,,,,,,,,,,
Market Cap(in B USD),2066.9400,2913.2800,2255.9700,1304.7600,748.5400,868.8700,617.5900,586.8600,647.3600,504.7900,500.6100,376.4000,296.8900,189.8000
Revenue,394328.0000,365817.0000,274515.0000,260174.0000,265595.0000,229234.0000,215639.0000,233715.0000,182795.0000,170910.0000,156508.0000,108249.0000,65225.0000,42905.0000
Gross Profit,170782.0000,152836.0000,104956.0000,98392.0000,101839.0000,88186.0000,84263.0000,93626.0000,70537.0000,64304.0000,68662.0000,43818.0000,25684.0000,17222.0000
Net Income,99803.0000,94680.0000,57411.0000,55256.0000,59531.0000,48351.0000,45687.0000,53394.0000,39510.0000,37037.0000,41733.0000,25922.0000,14013.0000,8235.0000
Earning Per Share,6.1100,5.6100,3.2800,2.9700,2.9800,2.3025,2.0775,2.3050,1.6125,1.4200,1.5775,0.9886,0.5411,0.3243
EBITDA,130541.0000,120233.0000,77344.0000,76477.0000,81801.0000,71501.0000,70529.0000,82487.0000,60449.0000,55756.0000,58518.0000,35604.0000,19412.0000,12474.0000
Share Holder Equity,50672.0000,63090.0000,65339.0000,90488.0000,107147.0000,134047.0000,128249.0000,119355.0000,111547.0000,123549.0000,118210.0000,76615.0000,47791.0000,31640.0000
Cash Flow from Operating,122151.0000,104038.0000,80674.0000,69391.0000,77434.0000,64225.0000,66231.0000,81266.0000,59713.0000,53666.0000,50856.0000,37529.0000,18595.0000,10159.0000
Cash Flow from Investing,-22354.0000,-14545.0000,-4289.0000,45896.0000,16066.0000,-46446.0000,-45977.0000,-56274.0000,-22579.0000,-33774.0000,-48227.0000,-40419.0000,-13854.0000,-17434.0000


In [ ]:
# Load the English language model
nlp = spacy.load("en_core_web_sm")
# Initialize the Matcher
matcher = PhraseMatcher(nlp.vocab)
# Define phrases for each row in the dataset (financial terms)
terms = list(financial_data.index)
term_patterns = [nlp(term) for term in terms]

# Add row patterns to the matcher
matcher.add("FinancialTerms", None, *term_patterns)

# Function to extract information from the dataset based on user question
def extract_information(question):
    doc = nlp(question)
    matches = matcher(doc)

    if matches:
        match_id, start, end = matches[0]
        matched_text = doc[start:end].text
        if matched_text in terms:
            # Extract relevant information from the dataset
            for col in financial_data.columns:
                if str(col) in question:
                    value = financial_data.loc[matched_text, col]
                    return f"The {matched_text} in {col} is {value}."
            # If no specific year mentioned, default to latest available year
            latest_year = financial_data.columns[0]  # Latest year is the first column
            latest_value = financial_data.loc[matched_text, latest_year]
            return f"The {matched_text} in {latest_year} is {latest_value}."
    else:
        return "I couldn't find any relevant information in the dataset."

# Function to handle the conversation
def financial_qa_system():
    print("Hello! I am Investo, Your Financial Question Answering Chatbot. How can I assist you today?")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Investo: Goodbye! Have a great day.")
            break
        elif user_input.lower() in ["hello", "hi", "hey", "hola", "greetings"]:
            print("Investo: Hello! How can I help you today?")
        else:
            response = extract_information(user_input)
            print("Investo:", response)

if __name__ == "__main__":
    financial_qa_system()

Hello! I am Investo, Your Financial Question Answering Chatbot. How can I assist you today?
You: hello
Investo: Hello! How can I help you today?
You: Revenue in 2019
Investo: The Revenue in 2019 is 260174.0.
You: bye
Investo: Goodbye! Have a great day.
